In [2]:
import mlflow
import os
import pandas as pd
import pickle
import numpy as np

Install MLflow - Q1 Answer

In [3]:
!mlflow --version 

mlflow, version 2.20.3


Download and preprocess the data - Q2

In [4]:
Homework_Path = os.getcwd()
Data_Path = "."

In [5]:
os.chdir(Data_Path)
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet
os.listdir()
os.chdir(Homework_Path)

--2025-03-12 05:08:57--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.160, 3.164.82.197, 3.164.82.40, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1.4M) [binary/octet-stream]
Saving to: ‘green_tripdata_2023-01.parquet.2’

green_tripdata_2023 100%[===================>]   1.36M  2.35MB/s    in 0.6s    

2025-03-12 05:08:59 (2.35 MB/s) - ‘green_tripdata_2023-01.parquet.2’ saved [1427002/1427002]

--2025-03-12 05:08:59--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.197, 3.164.82.160, 3.164.82.112, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.197|:443... connected.
HTTP request sent, aw

In [6]:
ls -ltrh green*.parquet | wc -l


4


Train a model with autolog - Q3

In [7]:
import mlflow
import mlflow.sklearn

mlflow.sklearn.autolog()



In [8]:
train_path = "./green_tripdata_2023-01.parquet"
val_path = "./green_tripdata_2023-02.parquet"
test_path = "./green_tripdata_2023-03.parquet"

df_train = pd.read_parquet(train_path)
df_val = pd.read_parquet(val_path)
df_test = pd.read_parquet(test_path)

df_train.head()
df_val.head()
df_test.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-03-01 00:25:10,2023-03-01 00:35:47,N,1.0,82,196,1.0,2.36,13.5,1.0,0.5,0.00,0.0,NaN,1.0,16.00,2.0,1.0,0.00
1,2,2023-03-01 00:14:29,2023-03-01 00:25:04,N,1.0,7,7,1.0,0.78,-6.5,-1.0,-0.5,0.00,0.0,NaN,-1.0,-9.00,3.0,1.0,0.00
2,2,2023-03-01 00:14:29,2023-03-01 00:25:04,N,1.0,7,7,1.0,0.78,6.5,1.0,0.5,0.00,0.0,NaN,1.0,9.00,3.0,1.0,0.00
3,2,2023-02-28 22:59:46,2023-02-28 23:08:38,N,1.0,166,74,1.0,1.66,11.4,1.0,0.5,2.78,0.0,NaN,1.0,16.68,1.0,1.0,0.00
4,2,2023-03-01 00:54:03,2023-03-01 01:03:14,N,1.0,236,229,1.0,3.14,15.6,1.0,0.5,4.17,0.0,NaN,1.0,25.02,1.0,1.0,2.75


In [9]:
import datetime
from sklearn.feature_extraction import DictVectorizer

def create_dataframe(filepath):
    df = pd.read_parquet(train_path)
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    df['pickup_day'] = df.lpep_pickup_datetime.apply(lambda x: x.weekday())
    df['pickup_hour'] = df.lpep_pickup_datetime.apply(lambda x: x.hour)
    df['pickup_min'] = df.lpep_pickup_datetime.apply(lambda x: x.minute)

    
    chosen_cols = ['duration','PULocationID','DOLocationID','trip_distance','pickup_day','pickup_hour','pickup_min']
    df = df[chosen_cols]

    return df

def preprocess_dataframe(df: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']

    numerical = ['trip_distance','pickup_day','pickup_hour','pickup_min']
    
    dicts = df[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

def dump_pickle(obj, filename: str):
    with open(filename, "wb") as f_out:
        return pickle.dump(obj, f_out)

In [10]:
dv = DictVectorizer()

df_train = create_dataframe(train_path)
df_val = create_dataframe(val_path)
df_test = create_dataframe(test_path)

X_train, dv = preprocess_dataframe(df_train,dv, fit_dv=True)
X_val, _ = preprocess_dataframe(df_val,dv, fit_dv=False)
X_test, _ = preprocess_dataframe(df_test,dv, fit_dv=False)

print(f"X_train contains: {np.shape(X_train)[0]} entries, and {np.shape(X_train)[1]} columns")
print(f"X_val contains: {np.shape(X_val)[0]} entries, and {np.shape(X_val)[1]} columns")
print(f"X_test contains: {np.shape(X_test)[0]} entries, and {np.shape(X_test)[1]} columns")

X_train contains: 65946 entries, and 5705 columns
X_val contains: 65946 entries, and 5705 columns
X_test contains: 65946 entries, and 5705 columns


In [12]:
y_train = df_train['duration']
y_val = df_val['duration']
y_test = df_test['duration']


dump_pickle(dv, os.path.join(Data_Path, "dv.pkl"))
dump_pickle((X_train, y_train), os.path.join(Data_Path, "train.pkl"))
dump_pickle((X_val, y_val), os.path.join(Data_Path, "val.pkl"))
dump_pickle((X_test, y_test), os.path.join(Data_Path, "test.pkl"))


In [13]:
import os
import pickle
import mlflow
import mlflow.sklearn


mlflow.sklearn.autolog()



from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np


def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

data_path = "."

X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

mlflow.sklearn.autolog()
with mlflow.start_run():
    rf = RandomForestRegressor(max_depth=10, random_state=0)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    print("RMSE:", rmse)


2025/03/12 05:10:10 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'
2025/03/12 05:10:38 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: When an mlflow-artifacts URI was supplied, the tracking URI must be a valid http or https URI, but it was currently set to file:///workspaces/MLOps/mlruns. Perhaps you forgot to set the tracking URI to the running MLflow server. To set the tracking URI, use either of the following methods:
1. Set the MLFLOW_TRACKING_URI environment variable to the desired tracking URI. `export MLFLOW_TRACKING_URI=http://localhost:5000`
2. Set the tracking URI programmatically by calling `mlflow.set_tracking_uri`. `mlflow.set_tracking_uri('http://localhost:5000')`


RMSE: 4.877393542248552


Launch the tracking server locally -Q4

In [14]:
database_uri = "sqlite:///mlflow.db"
experiment_name = "nyc-taxi-ride-prediction-single"

mlflow.set_tracking_uri(database_uri) 
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='/workspaces/MLOps/mlruns/1', creation_time=1741584546996, experiment_id='1', last_update_time=1741584546996, lifecycle_stage='active', name='nyc-taxi-ride-prediction-single', tags={}>

Tune model hyperparameters -Q5


In [15]:
!pip install hyperopt

In [16]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials    
from hyperopt.pyll import scope

search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 100, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 20, 1)),
        'criterion' : hp.choice('criterion', ['squared_error', 'poisson']),
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 10, 1)),
        'random_state': 42
    }

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model","RandomForestRegressor")
        mlflow.set_tag("developer","Marcus")
        mlflow.log_param("train-data-path",train_path)
        mlflow.log_param("val-data-path",val_path)
        mlflow.log_params(params)
        
        
        rf = RandomForestRegressor(**params)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse",rmse)
    return {'loss': rmse, 'status': STATUS_OK}

In [17]:
rstate = np.random.default_rng(42)
mlflow.sklearn.autolog()
best_result = fmin(
    fn = objective,
    space = search_space, 
    algo = tpe.suggest, 
    max_evals = 100,
    trials = Trials(), 
    rstate=rstate
)

  0%|                                                                                                                         | 0/100 [00:00<?, ?trial/s, best loss=?]

2025/03/12 05:11:00 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'

2025/03/12 05:11:43 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp7qgkaiw7/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 



  1%|▉                                                                                             | 1/100 [00:46<1:17:27, 46.95s/trial, best loss: 4.792555071263195]

2025/03/12 05:11:47 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'

2025/03/12 05:11:50 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp_ezc52ua/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 



  2%|█▉                                                                                              | 2/100 [00:52<36:27, 22.32s/trial, best loss: 4.792555071263195]

...

 98%|███████████████████████████████████████████████████████████████████████████▍ | 98/100 [49:00<00:56, 28.29s/trial, best loss: 3.9843860803819906]

2025/03/12 06:00:01 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'

2025/03/12 06:00:13 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpum9pqp63/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 



 99%|████████████████████████████████████████████████████████████████████████████▏| 99/100 [49:14<00:24, 24.07s/trial, best loss: 3.9843860803819906]

2025/03/12 06:00:15 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'

2025/03/12 06:00:18 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp1h1fn5du/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 



100%|████████████████████████████████████████████████████████████████████████████| 100/100 [49:19<00:00, 29.59s/trial, best loss: 3.9843860803819906]


In [ ]:
Promote the best model to the model registry -Q6

In [20]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment_id = "1"  


runs = client.search_runs(
    experiment_ids=[experiment_id],
    filter_string="metrics.rmse >= 0",
    order_by=["metrics.rmse ASC"],
    max_results=5
)


best_run = runs[0]
best_run_id = best_run.info.run_id
best_rmse = best_run.data.metrics["rmse"]

print("Best run ID:", best_run_id)
print("Best rmse:", best_rmse)

from mlflow import MlflowClient

client = MlflowClient()
client.create_registered_model("sk-learn-random-forest-reg-model")


Best run ID: 45c8cdfaff724c96bb232143c91ee850
Best rmse: 3.9843860803819906


<RegisteredModel: aliases={}, creation_timestamp=1741759935912, description=None, last_updated_timestamp=1741759935912, latest_versions=[], name='sk-learn-random-forest-reg-model', tags={}>